# Model evaluation

Evaluate the model performance and save the results in csv format.

In [1]:
from fastai.vision.all import *
import sklearn.metrics as skm
import pathlib
import glob

## Setup

In [2]:
# Load model
def get_multi_y(x): return parent_label(x).split("_")
model = load_learner("output/20230102_trained-model.pkl")

# Load datasets
dblock = DataBlock(blocks = (ImageBlock, MultiCategoryBlock),
                   get_items = get_image_files,
                   get_y = get_multi_y,
                   splitter = GrandparentSplitter(train_name="train", valid_name="valid")
                   )

dls = dblock.dataloaders("training/high-pass")

## Prediction on the validation dataset

In [3]:
# Create lists to store model predictions and the corresponding target labels
preds_class = []
preds_num = []
targs = []

# Use the trained model to predict the validation dataset
for i, item in enumerate(dls.valid.items):
    with model.no_bar(): pred_class, _, pred_num = model.predict(item)
    
    preds_class.append(pred_class)
    preds_num.append(pred_num.numpy())
    targs.append(dls.valid_ds[i][1].numpy())
    
# Convert the lists to numpy.ndarray for later use
preds_num = np.array(preds_num)
targs = np.array(targs)

Save model metrices (precision, recall, and f1-score) in a csv file.

In [4]:
metrics = skm.classification_report(targs[:,:5], preds_num[:,:5]>0.5, 
                          target_names=dls.vocab[:5], output_dict=True, zero_division = 0.0)

metrics_df = pd.DataFrame(metrics)

metrics_df.to_csv("output/metrics-results.csv")

Write out file names, corresponding model predictions, and the target labels.

In [5]:
preds_df = pd.DataFrame({
    "file": [o.name for o in dls.valid.items],
    "preds": ["_".join(classes) for classes in preds_class],
    "label": [o.parent.name for o in dls.valid.items]
})

preds_df[0:10]  # check

,file,preds,label
0,20210628_070000_0_Kyodai_Kyoto.png,background,background
1,20210802_210000_2_Kyodai_Kyoto.png,background,background
2,20210805_170000_1_Kyodai_Kyoto.png,background,background
3,20210813_190000_3_Kyodai_Kyoto.png,background,background
4,20210817_070000_9_Kyodai_Kyoto.png,background,background
5,20210903_230000_8_Kyodai_Kyoto.png,background,background
6,20210905_100000_5_Kyodai_Kyoto.png,background,background
7,20210908_160000_11_Kyodai_Kyoto.png,background,background
8,20220303_030000_0_Uryu_Kyoto.png,background,background
9,20220309_110000_0_Uryu_Kyoto.png,background,background


In [6]:
# Save
preds_df.to_csv("output/validation-results.csv") 